## MicroLearning -> [VMLPS00](https://confluence.gft.com/display/APACD/MicroLearnings)

> Purpose of this MicroLearning is to learn how Vault treats those PIB requests which do not have a valid client_id. client_id is Posting API Client ID. It is critical and mandatory information in the PIB using which Vault will route the PIB response to the right kafka response topic which corresponds to that Posting API Client. If a valid client_id is not provided, Vault's postings processor does not process that request at all, and sends it to postings DLQ. The header of the message written to the DLQ contains the information on why that message has been written to the DLQ and could not be processed by Vault.

#### Set up a Kafka Producer

In [ ]:
import json
import os
from dataclasses import dataclass
from typing import Dict, Callable, Any
from kafka import KafkaConsumer, KafkaProducer
import requests
import logging

CONTENT_TYPE = 'application/json'

logging.basicConfig(level=logging.INFO)
KAFKA_ENDPOINT = 'bootstrap.kafka.partner-eph-6.tmachine.io:443'
CORE_API_URL = "https://core-api.partner-eph-6.tmachine.io"
WORKFLOW_API_URL = "https://workflows-api.partner-eph-6.tmachine.io"
X_AUTH_TOKEN = "A0006786022557907328897!rI1MCYa35TdEFt3kka7xh5edAoXEHfXGzntcA4vSxAseR+Cu+rseyz+j9Ql4WffZD8IsAZ9DUKDttPlqvSNsrfZd6To="

PIB_URL_BATCH_GET = "/v1/posting-instruction-batches:batchGet"
PIB_TOPIC = 'vault.api.v1.postings.posting_instruction_batch.created'
PIB_REQUEST_TOPIC = 'vault.core.postings.requests.v1'
GET_PAC = "/v1/postings-api-clients"

class TMApiClient:
    @dataclass
    class TMConnectionDetails:
        core_api_url: str
        workflow_api_url: str
        token: str
        kafka_url: str
        kafka_security_protocol: str = "SSL"
        content_type: str = CONTENT_TYPE

    def __init__(self, conn: TMConnectionDetails):
        self.connection_details = conn
        self.default_headers = {
            'Content-Type': conn.content_type,
            'X-Auth-Token': conn.token
        }

    def call_core_api(self, endpoint: str, query_params: Dict[str, str], method: str = 'get', body: Any = None,
                      headers: Dict[str, str] = {}):
        return self.__call_api(url=self.connection_details.core_api_url + endpoint,
                               query_params=query_params,
                               method=method,
                               body=body,
                               extra_headers=headers)

    def call_workflow_api(self, endpoint: str, query_params: Dict[str, str], method: str = 'get', body: Any = None,
                          headers: Dict[str, str] = {}):
        return self.__call_api(url=self.connection_details.workflow_api_url + endpoint,
                               query_params=query_params,
                               method=method,
                               body=body,
                               extra_headers=headers)

    def __call_api(self, url: str, query_params: Dict[str, str], method: str, body: Any, extra_headers: Dict[str, str]):
        response = requests.request(method=method,
                                    url=url,
                                    params=query_params,
                                    data=body,
                                    headers={**self.default_headers, **extra_headers}
                                    )
        if not response.ok:
            raise Exception('Failed getting response for request %s, %s %s' % (
                str(response.request), response.status_code, response.text))
        return response.json()

    def publish_kafka_message(self, topic: str, key: str, value: Any) -> None:
        """
        Publish message to particular kafka topic.
        :param topic: Topic to where send the message
        :param key: Key of the message converted to bytes
        :param value: Value of the message, that can be converted to json and the to bytes
        :return: None
        """
        producer = KafkaProducer(bootstrap_servers=[self.connection_details.kafka_url],
                                 security_protocol=self.connection_details.kafka_security_protocol,
                                api_version=(0, 8, 2))

        producer.send(topic, key=key.encode('utf-8'), value=json.dumps(value).encode('utf-8'))

    def subscribe_to_kafka_topic(self,
                                 topic: str,
                                 group_id: str = str(os.getpid()),
                                 offset: str = 'earliest',
                                 callback: Callable[[str, str], None] = lambda *args: None,
                                 ) -> None:
        """
        Subscribe particular kafka topic to receive messages in json format encoded in utf-8
        :param topic: topic name to subscribe the consumer
        :param group_id: configure group id of consumers. Default value is process Id.
        :param offset: configure the offset for consumer between earliest,latest or exact offset value.
        Default value is earliest.
        :param callback: function that consumes message as arguments of key and value.
        :return: None
        """
        consumer = KafkaConsumer(topic,
                                 group_id=group_id, auto_offset_reset=offset,
                                 bootstrap_servers=[self.connection_details.kafka_url],
                                 security_protocol=self.connection_details.kafka_security_protocol,
                                 api_version=(0, 8, 2))
        logging.debug("Consumer for topic {} has started".format(topic))
        for record in consumer:
            logging.debug("%s:%d:%d: key=%s value=%s" % (record.topic, record.partition,
                                                         record.offset, record.key,
                                                         record.value))
            callback(record.key.decode('utf8'), json.loads(record.value.decode('utf8').replace("'", '"')))

details = TMApiClient.TMConnectionDetails(core_api_url=CORE_API_URL,
                                             workflow_api_url=WORKFLOW_API_URL,
                                             token=X_AUTH_TOKEN,
                                             kafka_url=KAFKA_ENDPOINT)

#### Publish a posting without the client_id

> If the client_id field in the PostingInstructionBatch is invalid. This could be because it is either empty, or the specified client_id has not been registered with the Postings API. Then, the PIB is not  processed byt Vault at all and a message is published to the DLQ because there was a problem determining which response topic to stream the response to. Read here about: [Postings DLQ](https://docs.thoughtmachine.net/vault-core/4-6/EN/api/postings_api/#dlqs)

In [ ]:
client = TMApiClient(details)
request = json.loads("""{
"request_id":"test",
"posting_instruction_batch":{
  "client_batch_id":"test",
  "posting_instructions":[
     {
        "client_transaction_id":"f9ea38b6-59a8-497d-9a43-dd907adb28d6",
        "instruction_details":null,
        "override":null,
        "transaction_code":null,
        "outbound_authorisation":null,
        "inbound_authorisation":null,
        "authorisation_adjustment":null,
        "settlement":null,
        "release":null,
        "inbound_hard_settlement":{
            "amount":"21",
            "denomination":"USD",
            "target_account":{
                "account_id":"40d78ae3-24f8-5393-251d-6ac5811c0433"
            },
            "internal_account_id":"migration_sample",
            "advice":true

        }
     }
  ],
  "batch_details":{
     "force_override":"true"
  }
}}
""")

client.publish_kafka_message(PIB_REQUEST_TOPIC, "40d78ae3-24f8-5393-251d-6ac5811c0433", request)

#### Confirm that the above posted PIB went to Postings DLQ topic i.e. 'vault.core.postings.requests.dlq.v1' 

> Vault would have sent the above Posting to DLQ topic because the client_id was not provided in the request body. A background kafka consumer has consumed all the DLQs and inserted them into a locally running mondoDB instance

In [ ]:
from pymongo import MongoClient
from pprint import pprint

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['vault_events']
collection = db['postings_dlq']

# Define the query
query = { "posting_instruction_batch.posting_instructions.client_transaction_id": "f9ea38b6-59a8-497d-9a43-dd907adb28d6" }

# Fetch the documents
result = collection.find(query)

# Iterate over the fetched documents
for document in result:
    pprint(document)

# Close the MongoDB connection
client.close()


#### Confirm that the above posted PIB was added to postings requests topic i.e. 'vault.core.postings.requests.v1'

In [ ]:
from pymongo import MongoClient
from pprint import pprint

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['vault_events']
collection = db['postings_requests']

# Define the query
query = { "posting_instruction_batch.posting_instructions.client_transaction_id": "f9ea38b6-59a8-497d-9a43-dd907adb28d6" }

# Fetch the documents
result = collection.find(query)

# Iterate over the fetched documents
for document in result:
    pprint(document)

# Close the MongoDB connection
client.close()
